In [1]:
import pandas as pd

NROWS = None

title_vec = pd.read_csv('hn_title_norm_vec.csv', nrows=NROWS)

In [11]:
origin = pd.read_csv('/Users/Shared/data/HN_posts_year_to_Sep_26_2016.csv')
origin = origin[['id', 'num_points']]
data = pd.merge(title_vec, origin, how='inner', on=['id'])

In [12]:
'''
sentence_length = pd.read_csv('sentence_length.csv', header=None, )
sentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)
data = pd.merge(data, sentence_length, how='inner', on=['id'])
'''

"\nsentence_length = pd.read_csv('sentence_length.csv', header=None, )\nsentence_length.rename(columns={0: 'id', 1:'length'}, inplace=True)\ndata = pd.merge(data, sentence_length, how='inner', on=['id'])\n"

In [13]:
GOOD_THRESHOLD = 100

data['good'] = data['num_points'].apply(lambda x: 1 if x >= GOOD_THRESHOLD else 0)
print(data['good'].describe())

count    293119.000000
mean          0.039370
std           0.194473
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: good, dtype: float64


In [14]:
data = data.drop(['id', 'num_points'], 1)

In [15]:
train = data.sample(frac=0.8)
test = data.drop(train.index)

In [16]:
import numpy as np
good = train[train['good'] == 1]
bad = train[train['good'] == 0].head(good.shape[0])
trainer = good.append(bad)
train = trainer.reindex(np.random.permutation(trainer.index))

In [53]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Activation, Bidirectional, Dropout
from keras.models import Model, Sequential

input_dim = 50
timesteps = 24
latent_dim = 200

predictor = Sequential()
#predictor.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(timesteps, input_dim), merge_mode='ave'))
#predictor.add(LSTM(100, input_shape=(timesteps, input_dim)))
predictor.add(Dense(1000, init='uniform', activation='sigmoid', input_shape=(timesteps * input_dim,)))
#predictor.add(Dropout(0.5))
predictor.add(Dense(1000, init='uniform', activation='sigmoid'))
#predictor.add(Dropout(0.5))
predictor.add(Dense(1000, init='uniform', activation='sigmoid'))
#predictor.add(Dropout(0.5))
predictor.add(Dense(1, init='uniform', activation='softmax'))

'''
decoded = RepeatVector(timesteps)(encoded)
#decoded = LSTM(input_dim, return_sequences=True)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)
'''

'\ndecoded = RepeatVector(timesteps)(encoded)\n#decoded = LSTM(input_dim, return_sequences=True)(encoded)\ndecoded = LSTM(input_dim, return_sequences=True)(decoded)\n\nsequence_autoencoder = Model(inputs, decoded)\nencoder = Model(inputs, encoded)\n'

In [54]:
from keras.utils.np_utils import to_categorical

label = ['good']

X = train.drop(label, axis=1)
Y = test.drop(label, axis=1)

X_label = train[label].as_matrix()
Y_label = test[label].as_matrix()
#X_label = to_categorical(train[label].as_matrix(), 2)
#Y_label = to_categorical(test[label].as_matrix(), 2)

X = X.as_matrix().reshape(train.shape[0], 24* 50)
Y = Y.as_matrix().reshape(test.shape[0], 24* 50)

In [55]:
predictor.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'precision', 'recall', 'fmeasure'])
predictor.fit(X, X_label, validation_data=(Y, Y_label), batch_size=1)

Train on 18594 samples, validate on 58624 samples
Epoch 1/10
18594/18594 [==============================] - 1221s - loss: 7.9712 - acc: 0.5000 - precision: 0.5000 - recall: 0.5000 - fmeasure: 0.5000 - val_loss: 15.3324 - val_acc: 0.0383 - val_precision: 0.0383 - val_recall: 0.0383 - val_fmeasure: 0.0383
Epoch 2/10
 1873/18594 [==>...........................] - ETA: 963s - loss: 7.8818 - acc: 0.5056 - precision: 0.5056 - recall: 0.5056 - fmeasure: 0.5056

KeyboardInterrupt: 